In [1]:
import pandas as pd
import dedupe
import os

In [2]:
# config
sheet_name = ["AllMasterDb", "Top6KMedsFromMasterDB", "DistributorMahaveer", "DistributorFMPL_unnati", "DistributorVMPL_unnati", "DistributorParshva"]
columns = {
    "AllMasterDb": ["name","Id","manufacturers","salt_composition"], 
    "Top6KMedsFromMasterDB": ["name"], 
    "DistributorMahaveer": ["ITEM CODE", "ITEM NAME"],
    "DistributorFMPL_unnati": ["Code","Item Name"], 
    "DistributorVMPL_unnati": ["Code","Item Name"],
    "DistributorParshva": ["Code","Descreption"]
}

standardized_columns = {
    "AllMasterDb": ["name","Id","manufacturers","salt_composition"], 
    "Top6KMedsFromMasterDB": ["name"], 
    "DistributorMahaveer": ["Id", "name"],
    "DistributorFMPL_unnati": ["Id", "name"], 
    "DistributorVMPL_unnati": ["Id", "name"],
    "DistributorParshva": ["Id", "name"]
}

output_columns = {
    "AllMasterDb": ["name","Id","manufacturers","salt_composition"], 
    "Top6KMedsFromMasterDB": ["dist_name"], 
    "DistributorMahaveer": ["Dist_Item_Code", "dist_name"],
    "DistributorFMPL_unnati": ["Dist_Item_Code", "dist_name"], 
    "DistributorVMPL_unnati": ["Dist_Item_Code", "dist_name"],
    "DistributorParshva": ["Dist_Item_Code", "dist_name"]
}

# Define the fields dedupe will pay attention to
#
# Notice how we are telling dedupe to use a custom field comparator
# for the 'Zip' field.
fields = [
    {'field' : 'name', 'type': 'String'}
]

In [3]:
def readData(file_location, sheet_name):
    """
    Read data from excel and create a dictionary of records,
    where key is a unique record ID and each column(specified in config) is dictionary
    """
    
    data_d = {}
    # nrows = maximum no of rows to get from excel file
    file = pd.read_excel(file_location, sheet_name=sheet_name, nrows=300000)
    print(f"No of Rows in Data is {len(file)}")
    c = columns[sheet_name]
    
    for i, row in enumerate(file[c].values):
        clean_row = [(k, v) for (k, v) in zip(standardized_columns[sheet_name], row)]
        data_d[i] = dict(clean_row)
    return data_d

In [47]:
def create_output(xml_file, left_sheet, right_sheet, output_file, linked_records)->None:
        
    left_data = pd.read_excel(xml_file, sheet_name=left_sheet, nrows=300000)[columns[left_sheet]]
    left_data.columns = output_columns[left_sheet]
    right_data = pd.read_excel(xml_file, sheet_name=right_sheet, nrows=300000)[columns[right_sheet]]
    right_data.columns = output_columns[right_sheet]
    
    left_idx = []
    right_idx = []
    similarity = []
    for(left, right), sim in linked_records:
        left_idx.append(left)
        right_idx.append(right)
        similarity.append(sim)
    
    dataframe = left_data.iloc[left_idx].to_dict("list")
    dataframe.update(right_data.iloc[right_idx].to_dict("list"))
    dataframe["resemblance"] = similarity
    
    df = pd.DataFrame(data = dataframe)
    
    df = df.sort_values('resemblance', ascending=False).drop_duplicates(['dist_name'])

    df.to_excel(output_file, sheet_name=right_sheet, index=False)

In [48]:
def TrainingDedupe(left_sheet, right_sheet):
    
    output_file = f'Temp/{left_sheet}_{right_sheet}_output.xlsx'
    settings_file = 'Temp/csv_example_learned_settings'
    training_file = 'Temp/csv_example_training.json'
    xml_file = "Data/ProductListMasterDB&Distributors.xlsx"
    
    print('importing data ...')
    data_left = readData(xml_file, left_sheet)
    data_right = readData(xml_file, right_sheet)

    # Training

    if os.path.exists(settings_file):
        print('reading from', settings_file)
        with open(settings_file, 'rb') as f:
            linker = dedupe.StaticRecordLink(f)

    else:

        # Create a new deduper object and pass our data model to it.
        linker = dedupe.RecordLink(fields)

        # If we have training data saved from a previous run of dedupe,
        # look for it and load it in.
        # __Note:__ if you want to train from scratch, delete the training_file
        if os.path.exists(training_file):
            print('reading labeled examples from ', training_file)
            with open(training_file, 'rb') as f:
                linker.prepare_training(data_left, data_right, f, sample_size=15000)
        else:
            linker.prepare_training(data_left, data_right, sample_size=15000)


        # ## Active learning
        # Dedupe will find the next pair of records
        # it is least certain about and ask you to label them as duplicates
        # or not.
        # use 'y', 'n' and 'u' keys to flag duplicates
        # press 'f' when you are finished
        print('starting active labeling...')

        dedupe.console_label(linker)

        linker.train()

        # When finished, save our training away to disk
        with open(training_file, 'w') as tf :
            linker.write_training(tf)

        # Save our weights and predicates to disk.  If the settings file
        # exists, we will skip all the training and learning next time we run
        # this file.
        with open(settings_file, 'wb') as sf :
            linker.write_settings(sf)

    
    # Clustering

    # `partition` will return sets of records that dedupe
    # believes are all referring to the same entity.

    print('clustering...')
    linked_records = linker.join(data_left, data_right, 0.0, constraint= "many-to-many")
    print('# Matched sets', len(linked_records))
    
    # Writing Results
    create_output(xml_file, left_sheet, right_sheet, output_file, linked_records)
    

In [49]:
TrainingDedupe(sheet_name[0], sheet_name[2])

importing data ...
No of Rows in Data is 283734
No of Rows in Data is 10438
reading from Temp/csv_example_learned_settings
clustering...
# Matched sets 96498
